In [1]:
!python --version

Python 3.12.3


In [2]:
!pip install fastapi uvicorn transformers sentencepiece torch pillow requests


   ---------------------------------------- 0.0/95.6 kB ? eta -:--:--
   ---------------------------------------- 95.6/95.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/66.4 kB ? eta -:--:--
   ---------------------------------------- 66.4/66.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 15.9 MB/s eta 0:00:01
   ------------------------- -------------- 0.7/1.1 MB 8.5 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 7.0 MB/s eta 0:00:01
   ---------------------------------- ----- 0.9/1.1 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/73.0 kB ? eta -:--:--
   ---------------------------------------- 73.0/73.0 kB 2.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install fastapi uvicorn transformers sentencepiece torch langdetect



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from torchvision import transforms
from PIL import Image
import torch

app = FastAPI()

# 1. Load your model
MODEL_PATH = "all_crops_hybrid_best_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the same architecture used during training
# Example placeholder—replace with your CNN+ViT definition
from your_model_definition import HybridModel  # <-- Import your model class

model = HybridModel()
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# 2. Preprocessing (adjust to your training config)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 3. Define your label map
CLASS_NAMES = [
    "banana_bacterial_wilt", "cashew_anthracnose",
    "coconut_leaf_spot", "coffee_rust",
    "mango_powdery_mildew", "pepper_leaf_spot",
    "paddy_blast", "wheat_rust", "potato_early_blight",
    # ... add all classes used during training
]

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        image = Image.open(file.file).convert("RGB")
        img_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)
            disease_name = CLASS_NAMES[predicted.item()]

        return JSONResponse(content={"predicted_disease": disease_name})
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

@app.get("/")
def root():
    return {"message": "Plant Disease Detection API is running"}


ModuleNotFoundError: No module named 'your_model_definition'